In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.utils as vutils




In [ ]:
% load_ext autoreload 
% autoreload 2

In [ ]:
from modules_tied import VAE
from modules_tied import NetD
from modules_tied import Aux
from modules_tied import loss_function

In [ ]:
bsz = 100
criterion = nn.BCELoss()


In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)


In [ ]:
netG = VAE()
netD = NetD()
#aux = Aux()
print(netG)

In [ ]:
optimizerD = optim.Adam(netD.parameters(), lr=1e-3)
optimizerG = optim.Adam(netG.parameters(), lr=1e-3)
#optimizer_aux = optim.Adam(aux.parameters(), lr=1e-4)

In [ ]:
input = torch.FloatTensor(bsz,28,28)
label = torch.FloatTensor(bsz)
real_label=1
fake_label=0
USE_CUDA=1
lamb = 1e-2
l1dist = nn.PairwiseDistance(1)
l2dist = nn.PairwiseDistance(2)
LeakyReLU = nn.LeakyReLU(0)

if(USE_CUDA):
    netG=netG.cuda()
    netD=netD.cuda()
    #aux = aux.cuda()
    #criterion=criterion.cuda()
    input,label=input.cuda(), label.cuda()
    l1dist = l1dist.cuda()
    l2dist = l2dist.cuda()
    LeakyReLU = LeakyReLU.cuda()

In [ ]:
def get_direct_gradient_penalty(netD, x, gamma, cuda):
    _,output = netD(x)
    gradOutput = torch.ones(output.size()).cuda() if cuda else torch.ones(output.size())
    
    gradient = torch.autograd.grad(outputs=output, inputs=x, grad_outputs=gradOutput, create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradientPenalty = (gradient.norm(2, dim=1)).mean() * gamma
    
    return gradientPenalty


In [ ]:
from train import run_trainer

In [ ]:
run_trainer(netG,netD,optimizerG,optimizerD,train_loader,bsz)